In [50]:
#If we want to paint the nodes such that no two adjacent 
#nodes are painted with the same color, how many colors do we need in minimum?

#max(X(i,c), y) J = y
#such that:
#          (X(i,c) + X(j,c))L(i,j) <= 1
#          y >= c * X(i,c)
#          sum over c (X(i,c)) = 1

In [51]:
import matplotlib.pyplot as plt
#networkx for drawing graph plots
import networkx as nx
#Define graph
G = nx.grid_2d_graph(5,5)
#pos for position of the nodes
pos = nx.spring_layout(G, iterations=100)

In [52]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd

In [53]:
model = AbstractModel()
#Number of nodes
model.N = Param(mutable=True)
#Set of nodes
model.i = RangeSet(1, model.N)
model.j = Set(initialize=model.i)
#Set of colors where maximum will be the total number of nodes
model.kolor = RangeSet(1, model.N)
#L is parameter showing if i is connected to j
#Connectivity Matrix
model.L = Param(model.i, model.j, initialize=0, mutable=True)
model.X = Var(model.i, model.kolor, domain=Binary)
model.y = Var(within=NonNegativeReals)

def rule_C1(model, i, j, c):
    return model.L[i,j]*(model.X[i,c] + model.X[j,c]) <= 1
model.C1 = Constraint(model.i, model.j, model.kolor, rule=rule_C1)

def rule_C2(model, i, c):
    return model.y >= c*model.X[i,c]
model.C2 = Constraint(model.i, model.kolor, rule=rule_C2)

def rule_C3(model, i):
    return sum(model.X[i,c] for c in model.kolor) == 1
model.C3 = Constraint(model.i, rule=rule_C3)
model.OF = Objective(expr=model.y, sense=minimize)

In [54]:
#glpk solver for linear models
opt = SolverFactory('glpk')
model.N = len(G.nodes)
#instance = model.create_instance('Ex36.dat')
instance = model.create_instance()


    mutable Param).  The linkage between this RangeSet and the original source
    data will be broken, so updating the data value in the future will not be
    reflected in this RangeSet.  To suppress this warning, explicitly convert
    the source data to a constant type (e.g., float, int, or immutable Param)
    mutable Param).  The linkage between this RangeSet and the original source
    data will be broken, so updating the data value in the future will not be
    reflected in this RangeSet.  To suppress this warning, explicitly convert
    the source data to a constant type (e.g., float, int, or immutable Param)


In [55]:
counter = 1
edges = []
coordinates = []
nodes = []
xvec = []
yvec = []
for nd in G.nodes:
    print(nd)
    nodes.append(counter)
    coordinates.append(nd)
    xvec.append(nd[0])
    yvec.append(nd[1])
    counter += 1
    
mygraph = {'nodes': nodes,
          'xy': coordinates,
          'x': xvec,
          'y': yvec}
df = pd.DataFrame(mygraph, columns=['nodes', 'xy', 'x', 'y'])

for i in instance.i:
    for j in instance.j:
        instance.L[i,j] = 0
        
for edg in G.edges:
    fn = edg[0]
    tn = edg[1]
    ind1 = (df['x'] == fn[0]) & (df['y'] == fn[1])
    f = df.loc[ind1, 'nodes']
    ind2 = (df['x'] == tn[0]) & (df['y'] == tn[1])
    t = df.loc[ind2, 'nodes']
    i = f.iloc[0]
    j = t.iloc[0]
    instance.L[i,j] = 1
    instance.L[j,i] = 1

(0, 0)
(0, 1)
(0, 2)
(0, 3)
(0, 4)
(1, 0)
(1, 1)
(1, 2)
(1, 3)
(1, 4)
(2, 0)
(2, 1)
(2, 2)
(2, 3)
(2, 4)
(3, 0)
(3, 1)
(3, 2)
(3, 3)
(3, 4)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)


In [56]:
results = opt.solve(instance)
for i in instance.i:
    for j in instance.j:
        if value(instance.L[i,j])>0:
            print((i,j), value(instance.L[i,j]))

(1, 2) 1
(1, 6) 1
(2, 1) 1
(2, 3) 1
(2, 7) 1
(3, 2) 1
(3, 4) 1
(3, 8) 1
(4, 3) 1
(4, 5) 1
(4, 9) 1
(5, 4) 1
(5, 10) 1
(6, 1) 1
(6, 7) 1
(6, 11) 1
(7, 2) 1
(7, 6) 1
(7, 8) 1
(7, 12) 1
(8, 3) 1
(8, 7) 1
(8, 9) 1
(8, 13) 1
(9, 4) 1
(9, 8) 1
(9, 10) 1
(9, 14) 1
(10, 5) 1
(10, 9) 1
(10, 15) 1
(11, 6) 1
(11, 12) 1
(11, 16) 1
(12, 7) 1
(12, 11) 1
(12, 13) 1
(12, 17) 1
(13, 8) 1
(13, 12) 1
(13, 14) 1
(13, 18) 1
(14, 9) 1
(14, 13) 1
(14, 15) 1
(14, 19) 1
(15, 10) 1
(15, 14) 1
(15, 20) 1
(16, 11) 1
(16, 17) 1
(16, 21) 1
(17, 12) 1
(17, 16) 1
(17, 18) 1
(17, 22) 1
(18, 13) 1
(18, 17) 1
(18, 19) 1
(18, 23) 1
(19, 14) 1
(19, 18) 1
(19, 20) 1
(19, 24) 1
(20, 15) 1
(20, 19) 1
(20, 25) 1
(21, 16) 1
(21, 22) 1
(22, 17) 1
(22, 21) 1
(22, 23) 1
(23, 18) 1
(23, 22) 1
(23, 24) 1
(24, 19) 1
(24, 23) 1
(24, 25) 1
(25, 20) 1
(25, 24) 1


In [57]:
print(value(instance.y))
for i in instance.i:
    for c in instance.kolor:
        if value(instance.X[i,c])>0:
            print((i,c), value(instance.X[i,c]))

2.0
(1, 1) 1.0
(2, 2) 1.0
(3, 1) 1.0
(4, 2) 1.0
(5, 1) 1.0
(6, 2) 1.0
(7, 1) 1.0
(8, 2) 1.0
(9, 1) 1.0
(10, 2) 1.0
(11, 1) 1.0
(12, 2) 1.0
(13, 1) 1.0
(14, 2) 1.0
(15, 1) 1.0
(16, 2) 1.0
(17, 1) 1.0
(18, 2) 1.0
(19, 1) 1.0
(20, 2) 1.0
(21, 1) 1.0
(22, 2) 1.0
(23, 1) 1.0
(24, 2) 1.0
(25, 1) 1.0


In [58]:
print("The solver returned a status of :" + str(results.solver.status))

from pyomo.opt import SolverStatus, TerminationCondition

if (results.solver.status == SolverStatus.ok) and (results.solver.termination_condition == TerminationCondition.optimal):
    print("this is feasible and optimal")
elif results.solver.termination_condition == TerminationCondition.infeasible:
    print("do something about it? or exit?")
else:
    #something else is wrong
    print(str(results.solver))

The solver returned a status of :ok
this is feasible and optimal


In [59]:
#instance.C1.pprint()

In [60]:
cvec=[]
for i in instance.i:
    for c in instance.kolor:
        if value(instance.X[i,c])>0:
            cvec.append(c)
l1 = nx.draw(G,pos,node_size=250,node_color=cvec,with_labels=False,width=6)
plt.show()

TypeError: '_AxesStack' object is not callable

<Figure size 432x288 with 0 Axes>